# Crest Factor vs. Phase Shift

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import util
from scipy.signal import fftconvolve as conv, butter, lfilter
from os.path import join
from matplotlib import cm
colors = cm.viridis.colors

## Check Stimuli

In [ ]:
src_dir = '../abx_software/webMUSHRA_c929877_20180814/configs/resources/stimuli'
suffix = '.wav'
file_names = ['castanets_phi000',
              'hotelcalifornia_phi000',
              'pnoise_phi000',
              'pnoise_ref',
              'square_a_phi000',
              'square_b_phi000']
fade_in, fade_out = 32, 32

filter_order = 2**16
half_length = int(filter_order / 2)
beta = 8.6
phase_angles = np.linspace(0, 2 * np.pi, endpoint=False, num=int(360/5))
phase_shifters = [util.constant_phase_shifter(filter_order, phi, beta=beta)[1]
                  for phi in phase_angles]

In [ ]:
fs = 44100
hpf_order = 2
f_cutoff = 12.5
b, a = butter(hpf_order, Wn=f_cutoff, btype='high', fs=fs)

In [ ]:
C = {}
C_hpf = {}
for fn in file_names:
    sig, fs = sf.read(join(src_dir, fn + suffix))
    sig = sig[:,0]
    crest_factor = np.zeros_like(phase_angles)
    for i, h in enumerate(phase_shifters):
        sig_shift = util.fade(conv(sig, h)[half_length:-half_length], fade_in, fade_out)
        crest_factor[i] = util.crest_factor(sig_shift, oversample=1)   # oversample=4 takes long time
    C[fn] = crest_factor

    # HPF
    sig = lfilter(b, a, sig)
    crest_factor = np.zeros_like(phase_angles)
    for i, h in enumerate(phase_shifters):
        sig_shift = util.fade(conv(sig, h)[half_length:-half_length], fade_in, fade_out)
        crest_factor[i] = util.crest_factor(sig_shift, oversample=1)
    C_hpf[fn] = crest_factor

In [ ]:
phase_angles_deg = np.rad2deg(phase_angles)
viridis = cm.get_cmap('viridis', len(C)).colors

fig, ax = plt.subplots(figsize=(10, 10))
for i, fn in enumerate(file_names):
    #ax.plot(phase_angles_deg, util.db(C[fn]), label=fn, c=viridis[i])
    #ax.plot(phase_angles_deg, util.db(C_hpf[fn]), ls='--', c=viridis[i])
    ax.plot(phase_angles_deg, util.db(C[fn]), label=fn, c='C'+str(i))
    ax.plot(phase_angles_deg, util.db(C_hpf[fn]), ls='--', c='C'+str(i))    
ax.set_xticks(np.arange(0, 360, 30))
ax.legend(loc='best')
ax.grid();
ax.set_xlabel(r'$\phi$ / deg')
ax.set_ylabel(r'Crest Factor / dB')
ax.set_title('line: original, dash line: high pass filtered');